# Example of DOV search methods for interpretations (formele stratigrafie)

## Use cases explained below
* Get 'formele stratigrafie' in a bounding box
* Get 'formele stratigrafie' with specific properties
* Get 'formele stratigrafie' in a bounding box based on specific properties
* Select 'formele stratigrafie' in a municipality and return date
* Get 'formele stratigrafie' based on fields not available in the standard output dataframe
* Get 'formele stratigrafie' data, returning fields not available in the standard output dataframe

In [46]:
%matplotlib inline
import inspect, sys
import warnings; warnings.simplefilter('ignore')

In [47]:
# check pydov path
import pydov

## Get information about the datatype 'Formele stratigrafie'

In [48]:
from pydov.search.interpretaties import FormeleStratigrafieSearch
itp = FormeleStratigrafieSearch()

A description is provided for the 'Formele stratigrafie' datatype:

In [49]:
itp.get_description()

'Een formele stratigrafie van een boring of een sondering is een lithostratigrafische interpretatie van een (gecodeerde) lithologische beschrijving op basis van een welbepaalde vastgelegde standaard per diepte-interval. DOV gebruikt de gangbare standaarden binnen Vlaanderen. U vindt een overzicht van de gebruikte standaarden op onze website. De formele stratigrafie wordt gebruikt om de dikte van het Quartair te identificeren, en om de onderliggende pakketten Neogene, Paleogene en/of oudere lagen te onderscheiden en te identificeren.'

The different fields that are available for objects of the 'Formele stratigrafie' datatype can be requested with the get_fields() method:

In [50]:
fields = itp.get_fields()

# print available fields
for f in fields.values():
    print(f['name'])

Opdrachten
pkey_sondering
diepte_laag_van
diepte_tot_m
Datum
Z_mTAW
Proefnummer
betrouwbaarheid_interpretatie
lid1
lid2
Type_proef
relatie_lid1_lid2
pkey_boring
Geldig_van
Proeffiche
gemeente
generated_id
Auteurs
pkey_interpretatie
diepte_laag_tot
y
x
Geldig_tot


You can get more information of a field by requesting it from the fields dictionary:
* *name*: name of the field
* *definition*: definition of this field
* *cost*: currently this is either 1 or 10, depending on the datasource of the field. It is an indication of the expected time it will take to retrieve this field in the output dataframe.
* *notnull*: whether the field is mandatory or not
* *type*: datatype of the values of this field

In [51]:
fields['Datum']

{'cost': 1,
 'definition': 'De datum waarop de formele stratigrafie werd beschreven.',
 'name': 'Datum',
 'notnull': False,
 'type': 'date'}

## Example use cases

### Get 'Formele stratigrafie' in a bounding box

Get data for all the 'Formele stratigrafie' interpretations that are geographically located within the bounds of the specified box.

The coordinates are in the Belgian Lambert72 (EPSG:31370) coordinate system and are given in the order of lower left x, lower left y, upper right x, upper right y.

In [52]:
from pydov.util.location import Within, Box

df = itp.search(location=Within(Box(153145, 206930, 153150, 206935)))
df.head()

[000/001] c


,pkey_interpretatie,pkey_boring,pkey_sondering,betrouwbaarheid_interpretatie,x,y,diepte_laag_van,diepte_laag_tot,lid1,relatie_lid1_lid2,lid2
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,NaN,onbekend,153147.0,206931.0,0.0,3.00,Q,T,Q
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,NaN,onbekend,153147.0,206931.0,3.0,14.05,U,T,Bc


The dataframe contains one 'Formele stratigrafie' interpretation where two layers ('laag') were identified. The available data are flattened to represent unique attributes per row of the dataframe.

Using the *pkey_interpretatie* field one can request the details of this interpretation in a webbrowser:

In [53]:
for pkey_interpretatie in set(df.pkey_interpretatie):
    print(pkey_interpretatie)

https://www.dov.vlaanderen.be/data/interpretatie/2000-001087


### Get 'Formele stratigrafie' with specific properties

Next to querying interpretations based on their geographic location within a bounding box, we can also search for interpretations matching a specific set of properties. For this we can build a query using a combination of the 'FormeleStratigrafie' fields and operators provided by the WFS protocol.

A list of possible operators can be found below:

In [54]:
[i for i,j in inspect.getmembers(sys.modules['owslib.fes'], inspect.isclass) if 'Property' in i]

['PropertyIsBetween',
 'PropertyIsEqualTo',
 'PropertyIsGreaterThan',
 'PropertyIsGreaterThanOrEqualTo',
 'PropertyIsLessThan',
 'PropertyIsLessThanOrEqualTo',
 'PropertyIsLike',
 'PropertyIsNotEqualTo',
 'PropertyIsNull',
 'SortProperty']

In this example we build a query using the *PropertyIsGreaterThan* and *PropertyIsEqualTo* operators to find all interpretations that are at least 20 m deep, that are deemed appropriate for a range of 1 km from a defined point:

In [55]:
from owslib.fes import And, PropertyIsGreaterThan, PropertyIsEqualTo
from pydov.util.location import WithinDistance, Point

query = And([PropertyIsEqualTo(propertyname='Betrouwbaarheid',
                              literal='goed'),
            PropertyIsGreaterThan(propertyname='diepte_tot_m',
                                 literal='20'),
           ])
            
df = itp.search(query=query, 
                location=WithinDistance(Point(153145, 206930), 1000))

df.head()

[000/007] ccccccc


,pkey_interpretatie,pkey_boring,pkey_sondering,betrouwbaarheid_interpretatie,x,y,diepte_laag_van,diepte_laag_tot,lid1,relatie_lid1_lid2,lid2
0,https://www.dov.vlaanderen.be/data/interpretat...,NaN,https://www.dov.vlaanderen.be/data/sondering/1...,goed,153898.0,207038.0,0.0,4.2,Q,T,Q
1,https://www.dov.vlaanderen.be/data/interpretat...,NaN,https://www.dov.vlaanderen.be/data/sondering/1...,goed,153898.0,207038.0,4.2,21.0,Bc,T,Bc
2,https://www.dov.vlaanderen.be/data/interpretat...,NaN,https://www.dov.vlaanderen.be/data/sondering/1...,goed,153898.0,207038.0,21.0,28.2,Bm,T,Bm
3,https://www.dov.vlaanderen.be/data/interpretat...,NaN,https://www.dov.vlaanderen.be/data/sondering/1...,goed,153956.0,207033.0,0.0,3.8,Q,T,Q
4,https://www.dov.vlaanderen.be/data/interpretat...,NaN,https://www.dov.vlaanderen.be/data/sondering/1...,goed,153956.0,207033.0,3.8,20.0,Bc,T,Bc


Once again we can use the *pkey_interpretatie* as a permanent link to the information of these interpretations:

In [56]:
for pkey_interpretatie in set(df.pkey_interpretatie):
    print(pkey_interpretatie)

https://www.dov.vlaanderen.be/data/interpretatie/2006-240080
https://www.dov.vlaanderen.be/data/interpretatie/2006-240081
https://www.dov.vlaanderen.be/data/interpretatie/2006-240082
https://www.dov.vlaanderen.be/data/interpretatie/2006-240083
https://www.dov.vlaanderen.be/data/interpretatie/2006-240085
https://www.dov.vlaanderen.be/data/interpretatie/2006-240086
https://www.dov.vlaanderen.be/data/interpretatie/2006-240153


### Get 'Formele stratigrafie' in a bounding box based on specific properties

We can combine a query on attributes with a query on geographic location to get the interpretations within a bounding box that have specific properties.

The following example requests the interpretations of boreholes only, within the given bounding box.

(Note that the datatype of the *literal* parameter should be a string, regardless of the datatype of this field in the output dataframe.)

In [57]:
from owslib.fes import PropertyIsEqualTo

query = PropertyIsEqualTo(
            propertyname='Type_proef',
            literal='Boring')

df = itp.search(
    location=Within(Box(153145, 206930, 154145, 207930)),
    query=query
    )

df.head()

[000/024] cccccccccccccccccccccccc


,pkey_interpretatie,pkey_boring,pkey_sondering,betrouwbaarheid_interpretatie,x,y,diepte_laag_van,diepte_laag_tot,lid1,relatie_lid1_lid2,lid2
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1984...,NaN,goed,153793.0,206970.0,0.0,4.5,Q,T,Q
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,NaN,onbekend,153217.0,207010.0,0.0,2.0,Q,T,Q
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,NaN,onbekend,153217.0,207010.0,2.0,72.0,Bc,T,Bc
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,NaN,onbekend,153174.0,207102.0,0.0,3.0,Q,T,Q
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,NaN,onbekend,153174.0,207102.0,3.0,6.5,Bc,T,Bc


We can look at one of the interpretations in a webbrowser using its *pkey_interpretatie*:

In [58]:
for pkey_interpretatie in set(df.pkey_interpretatie):
    print(pkey_interpretatie)

https://www.dov.vlaanderen.be/data/interpretatie/2000-000327
https://www.dov.vlaanderen.be/data/interpretatie/2000-007740
https://www.dov.vlaanderen.be/data/interpretatie/2000-007694
https://www.dov.vlaanderen.be/data/interpretatie/2000-007695
https://www.dov.vlaanderen.be/data/interpretatie/2000-023870
https://www.dov.vlaanderen.be/data/interpretatie/2000-007697
https://www.dov.vlaanderen.be/data/interpretatie/2000-007698
https://www.dov.vlaanderen.be/data/interpretatie/2000-007699
https://www.dov.vlaanderen.be/data/interpretatie/1999-007793
https://www.dov.vlaanderen.be/data/interpretatie/2000-023811
https://www.dov.vlaanderen.be/data/interpretatie/2000-023813
https://www.dov.vlaanderen.be/data/interpretatie/2000-023815
https://www.dov.vlaanderen.be/data/interpretatie/2000-023816
https://www.dov.vlaanderen.be/data/interpretatie/2000-001313
https://www.dov.vlaanderen.be/data/interpretatie/2000-007696
https://www.dov.vlaanderen.be/data/interpretatie/2000-001087
https://www.dov.vlaander

### Select 'Formele stratigrafie' in a municipality and return date

We can limit the columns in the output dataframe by specifying the *return_fields* parameter in our search.

In this example we query all the 'Formele stratigrafie' interpretations in a distance of 1 km and return their date:

In [59]:
df = itp.search(location=WithinDistance(Point(153145, 206930), 1000),
                return_fields=('Datum',))
df.head()

,Datum
0,1999-03-01
1,2000-05-20
2,2000-05-20
3,1999-03-01
4,2000-05-20


In [60]:
df.describe()

,Datum
count,40
unique,6
top,2000-05-20
freq,23


### Get 'Formele stratigrafie' based on fields not available in the standard output dataframe

To keep the output dataframe size acceptable, not all availabe WFS fields are included in the standard output. However, one can use this information to select interpretations as illustrated below.

For example, make a selection of the interpretations in municipality the of Antwerp, before 1/1/1990:

!*remark: mind that the municipality attribute is merely an attribute that is defined by the person entering the data. It can be ok, empty, outdated or wrong*!

In [61]:
from owslib.fes import And, PropertyIsEqualTo, PropertyIsLessThan

query = And([PropertyIsEqualTo(propertyname='gemeente',
                               literal='Antwerpen'),
             PropertyIsLessThan(propertyname='Datum', 
                                 literal='1990-01-01')]
            )
df = itp.search(query=query,
                return_fields=('pkey_interpretatie', 'Datum'))
df.head()

,pkey_interpretatie,Datum
0,https://www.dov.vlaanderen.be/data/interpretat...,1984-09-10


### Get 'Formele stratigrafie' data, returning fields not available in the standard output dataframe

As denoted in the previous example, not all available fields are available in the default output frame to keep its size limited. However, you can request any available field by including it in the *return_fields* parameter of the search:

In [62]:
query = PropertyIsEqualTo(
            propertyname='gemeente',
            literal='Leuven')

df = itp.search(query=query,
                return_fields=('pkey_interpretatie', 'pkey_boring', 'pkey_sondering',
                               'x', 'y', 'Z_mTAW', 'gemeente', 'Auteurs', 'Proefnummer'))

df.head()

,pkey_interpretatie,pkey_boring,pkey_sondering,x,y,Z_mTAW,Proefnummer,gemeente,Auteurs
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1998...,NaN,173055.7,177004.4,26.1,GEO-98/179-PB602o,Leuven,"De Ceukelaire, Marleen - MVG - Afdeling Natuur..."
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1905...,NaN,173473.0,174304.0,29.0,kb32d89e-B19,Leuven,"Claes, S. - Katholieke Universiteit Leuven (KUL)"
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1908...,NaN,172638.0,173884.0,23.0,kb32d89e-B107,Leuven,"Claes, S. - Katholieke Universiteit Leuven (KUL)"
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1966...,NaN,172189.0,170809.0,53.5,kb32d89e-B260,Leuven,"Gullentops, F. - Katholieke Universiteit Leuve..."
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1991...,NaN,174081.0,175579.0,19.0,kb32d89e-B337,Leuven,"Van Frausem, A. - Katholieke Universiteit Leuv..."


## Visualize results

Using Folium, we can display the results of our search on a map.

In [63]:
# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Proj, transform

In [64]:
# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    inProj = Proj(init='epsg:31370')
    outProj = Proj(init='epsg:4326')
    x2,y2 = transform(inProj, outProj, x1, y1)
    return x2, y2
df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y'])) 
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [65]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=12)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['Proefnummer'][loc]).add_to(marker_cluster)
fmap
